In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam


In [6]:
df= pd.read_csv('LSTM-data.csv')
df.head()

,Unnamed: 0,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
0,1738,2017-04-17,278.10,629.47,20636.92,18355.26,3222.17,1133.02,164.63,10.4037,1.23,2.165,52.73,55.36,1291.9,14.66,1
1,1737,2017-04-18,278.23,634.73,20636.92,18418.59,3196.71,1133.02,166.03,10.5386,1.23,2.165,52.73,55.36,1291.9,14.66,0
2,1736,2017-04-19,276.49,635.99,20523.28,18432.20,3170.69,1142.64,166.05,10.5056,1.23,2.165,52.33,54.89,1294.1,14.42,1
3,1735,2017-04-20,277.76,635.80,20404.49,18430.49,3172.10,1143.69,165.37,10.4108,1.24,2.165,50.62,52.93,1283.4,14.93,1
4,1734,2017-04-21,280.05,634.96,20578.71,18620.75,3173.15,1138.12,164.77,10.4008,1.24,2.165,50.27,52.99,1283.8,14.15,1


In [13]:
df_fi=df.drop(['Unnamed: 0','usd_krw', 'cny_krw','gold','wti','jpy_krw','vix'], axis=1)
df_fi.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,콜금리,국고채,brent,Target
0,2017-04-17,278.10,629.47,20636.92,18355.26,3222.17,1.23,2.165,55.36,1
1,2017-04-18,278.23,634.73,20636.92,18418.59,3196.71,1.23,2.165,55.36,0
2,2017-04-19,276.49,635.99,20523.28,18432.20,3170.69,1.23,2.165,54.89,1
3,2017-04-20,277.76,635.80,20404.49,18430.49,3172.10,1.24,2.165,52.93,1
4,2017-04-21,280.05,634.96,20578.71,18620.75,3173.15,1.24,2.165,52.99,1


In [15]:
# 데이터를 역순으로 정렬하여 전체 데이터의 70% 학습, 30% 테스트에 사용
df_fi = df_fi[::-1]
df_fi.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,콜금리,국고채,brent,Target
1738,2024-05-14,371.04,862.22,39431.64,38314.50,3146.29,3.581,3.429,83.36,0
1737,2024-05-13,370.98,854.43,39431.64,38179.46,3148.17,3.581,3.429,83.36,1
1736,2024-05-10,371.08,864.16,39387.76,38257.00,3155.65,3.581,3.429,83.88,0
1735,2024-05-09,368.84,870.15,39055.73,38070.50,3155.43,3.581,3.429,83.58,1
1734,2024-05-08,374.09,872.47,38883.94,38202.37,3128.48,3.581,3.429,82.99,0


In [17]:
# 패키지 임포트
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import MinMaxScaler

# 날짜 칼럼을 datetime 객체로 변환
df_fi['날짜'] = pd.to_datetime(df_fi['날짜'])
df_fi.set_index('날짜', inplace=True)

# 타깃 변수 만들어주기
# 단순히 다음 날 오르면 1, 하락하면 0
df_fi['Target'] = (df_fi['kospi200'].shift(-1) > df_fi['kospi200']).astype(int)


In [18]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 피처와 타겟을 분리합니다
features = df_fi.drop('Target', axis=1)
target = df_fi['Target']

# 피처 스케일링
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# LSTM 입력 형식으로 변환 (samples, time steps, features)
time_steps = 5  # 이전 5일의 데이터를 사용하여 예측

X = []
y = []

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(target.values[i])

X, y = np.array(X), np.array(y)


In [19]:
split_ratio = 0.7  # 훈련 세트 비율
split_index = int(split_ratio * len(X))

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=20))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [21]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
31/31 [==============================] - 7s 57ms/step - loss: 0.6948 - accuracy: 0.4794 - val_loss: 0.6903 - val_accuracy: 0.5514
Epoch 2/100
31/31 [==============================] - 0s 10ms/step - loss: 0.6907 - accuracy: 0.5351 - val_loss: 0.6892 - val_accuracy: 0.5514
Epoch 3/100
31/31 [==============================] - 0s 11ms/step - loss: 0.6915 - accuracy: 0.5155 - val_loss: 0.6879 - val_accuracy: 0.5514
Epoch 4/100
31/31 [==============================] - 0s 10ms/step - loss: 0.6923 - accuracy: 0.5093 - val_loss: 0.6884 - val_accuracy: 0.5514
Epoch 5/100
31/31 [==============================] - 0s 11ms/step - loss: 0.6914 - accuracy: 0.5165 - val_loss: 0.6878 - val_accuracy: 0.5514
Epoch 6/100
31/31 [==============================] - 0s 11ms/step - loss: 0.6908 - accuracy: 0.5113 - val_loss: 0.6874 - val_accuracy: 0.5514
Epoch 7/100
31/31 [==============================] - 0s 10ms/step - loss: 0.6891 - accuracy: 0.5433 - val_loss: 0.6879 - val_accuracy: 0.5514
Epoch 

In [22]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # 이진 분류 문제이므로 0.5를 기준으로 예측


17/17 [==============================] - 1s 5ms/step


In [23]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5431861804222649


----------------------------------------------------

In [24]:
# 피쳐 중요도 값을 딕셔너리 형태로 정의 (이미 제공된 값 사용)
feature_importance = {
    'dow': 0.325531,
    'ssec': 0.238900,
    'nikkei': 0.195132,
    'brent': 0.076208,
    'kosdaq': 0.006516,
    'cny_krw': -0.000277,
    'usd_krw': -0.020725,
    'gold': -0.030892,
    'wti': -0.032822,
    'jpy_krw': -0.172221,
    'vix': -0.223214
}

# 피쳐 중요도 값을 배열 형태로 정의
feature_importance_arr = np.array(list(feature_importance.values()))

# 피쳐 가중치 적용
for feature, importance in feature_importance.items():
    if feature in df.columns:
        df[feature] *= importance

df.head()

,Unnamed: 0,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
날짜,,,,,,,,,,,,,,,,
2024-05-14,0,371.04,5.618226,12836.221201,7476.385014,751.648681,-28.290661,-0.052223,-1.500510,3.581,3.429,-2.601143,6.352699,-72.379956,-3.035710,0
2024-05-13,1,370.98,5.567466,12836.221201,7450.034389,752.097813,-28.290661,-0.052278,-1.504729,3.581,3.429,-2.567337,6.352699,-73.104373,-3.035710,1
2024-05-10,2,371.08,5.630867,12821.936901,7465.164924,753.884785,-28.306205,-0.052552,-1.515820,3.581,3.429,-2.620508,6.392327,-72.296548,-2.832586,0
2024-05-09,3,368.84,5.669897,12713.850843,7428.772806,753.832227,-28.276983,-0.052389,-1.513581,3.581,3.429,-2.608364,6.369465,-71.740492,-2.901782,1
2024-05-08,4,374.09,5.685015,12657.927872,7454.504863,747.393872,-28.135431,-0.052303,-1.511050,3.581,3.429,-2.577512,6.324502,-71.799186,-2.953121,0


In [26]:
df=df.drop(['Unnamed: 0'], axis=1)
df.head()

,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
날짜,,,,,,,,,,,,,,,
2024-05-14,371.04,5.618226,12836.221201,7476.385014,751.648681,-28.290661,-0.052223,-1.500510,3.581,3.429,-2.601143,6.352699,-72.379956,-3.035710,0
2024-05-13,370.98,5.567466,12836.221201,7450.034389,752.097813,-28.290661,-0.052278,-1.504729,3.581,3.429,-2.567337,6.352699,-73.104373,-3.035710,1
2024-05-10,371.08,5.630867,12821.936901,7465.164924,753.884785,-28.306205,-0.052552,-1.515820,3.581,3.429,-2.620508,6.392327,-72.296548,-2.832586,0
2024-05-09,368.84,5.669897,12713.850843,7428.772806,753.832227,-28.276983,-0.052389,-1.513581,3.581,3.429,-2.608364,6.369465,-71.740492,-2.901782,1
2024-05-08,374.09,5.685015,12657.927872,7454.504863,747.393872,-28.135431,-0.052303,-1.511050,3.581,3.429,-2.577512,6.324502,-71.799186,-2.953121,0


In [27]:
# 피처와 타겟을 분리합니다
features = df.drop('Target', axis=1)
target = df['Target']

# 피처 스케일링
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# LSTM 입력 형식으로 변환 (samples, time steps, features)
time_steps = 5  # 이전 5일의 데이터를 사용하여 예측

X = []
y = []

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(target.values[i])

X, y = np.array(X), np.array(y)


In [28]:
split_ratio = 0.7  # 훈련 세트 비율
split_index = int(split_ratio * len(X))

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=20))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [30]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
31/31 [==============================] - 7s 42ms/step - loss: 0.6912 - accuracy: 0.5289 - val_loss: 0.6861 - val_accuracy: 0.5514
Epoch 2/100
31/31 [==============================] - 0s 15ms/step - loss: 0.6926 - accuracy: 0.5124 - val_loss: 0.6865 - val_accuracy: 0.5514
Epoch 3/100
31/31 [==============================] - 0s 14ms/step - loss: 0.6892 - accuracy: 0.5320 - val_loss: 0.6864 - val_accuracy: 0.5514
Epoch 4/100
31/31 [==============================] - 0s 15ms/step - loss: 0.6894 - accuracy: 0.5216 - val_loss: 0.6867 - val_accuracy: 0.5514
Epoch 5/100
31/31 [==============================] - 1s 19ms/step - loss: 0.6879 - accuracy: 0.5289 - val_loss: 0.6873 - val_accuracy: 0.5514
Epoch 6/100
31/31 [==============================] - 1s 26ms/step - loss: 0.6885 - accuracy: 0.5216 - val_loss: 0.6870 - val_accuracy: 0.5514
Epoch 7/100
31/31 [==============================] - 1s 26ms/step - loss: 0.6885 - accuracy: 0.5216 - val_loss: 0.6922 - val_accuracy: 0.5514
Epoch 

In [31]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # 이진 분류 문제이므로 0.5를 기준으로 예측


17/17 [==============================] - 1s 3ms/step


In [32]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5777351247600768
